In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as ss
import matplotlib.pyplot as plt
from pyntcloud import PyntCloud
from tqdm import tqdm

plt.rcParams["figure.figsize"] = (8, 6)

In [ ]:
def show_bar_plots(hists: np.ndarray):
    n, m = hists.shape
    x = np.arange(m)
    f, ax = plt.subplots(figsize=(16, 4))
    for i, y in enumerate(hists):
        width = 1 / n
        shift = (i + 0.5) * width - 0.5
        plt.bar(x + shift, y, width=width, align='center')
    plt.show()

In [ ]:
def show_distances(path, thr=1.5, k=0):
    df = pd.read_csv(path)
    dists = np.sort(df.values.flatten())[-k:]
    print('Number of good correspondences:', np.count_nonzero(dists < thr))
    plt.fill_between(np.arange(len(dists)), np.zeros_like(dists), dists, facecolor='blue', alpha=0.5)
    plt.hlines(thr, 0, len(dists) - 1, color='r')
    plt.show()

In [ ]:
def show_uniquenesses(path, a_max=300):
    df = pd.read_csv(path)
    dists = df.values.flatten()
    dists = np.clip(np.sort(dists), a_min=0, a_max=a_max)
    plt.hist(dists, bins='fd')
    plt.show()

In [ ]:
show_uniquenesses('data/debug/KilwaKisiwani_40r_KilwaKisiwani_43r_uniquenesses_src_random_01.csv')

In [ ]:
show_uniquenesses('data/debug/Hokuyo_4_Hokuyo_8_uniquenesses_src_random_01.csv')

In [ ]:
show_uniquenesses('data/debug/bun000_bun045_uniquenesses_src_random_01.csv')

In [ ]:
def display_colored(df, columns=[]):
    display(df.style.background_gradient(subset=['pcc'] + columns))

In [ ]:
columns=['descriptor','testname','rmse','r_err','t_err','pcd_err','voxel_size','normal_radius_coef','feature_radius_coef','pcc', 'correct_correspondences', 'correspondences', 'inliers', 'correct_inliers','filter','threshold','n_random']

In [ ]:
df = pd.read_csv('data/debug/test_results.csv')
df['pcc'] = df['correct_correspondences'] / df['correspondences']
df = df[['testname','pcc', 'correct_correspondences','threshold']]
display_colored(df)

In [ ]:
df = pd.read_csv('data/debug/test_results.csv')
df['pcc'] = df['correct_correspondences'] / df['correspondences']
df = df[df['filter'] == 'random']
df = df[['testname','pcc', 'correct_correspondences','threshold']]
display_colored(df)

### FPFH

In [ ]:
df = pd.read_csv('data/debug/fpfh_results_02.csv')
df['pcc'] = df['correct_correspondences'] / df['correspondences']
df_bun = df[df['testname'] == "bun000_bun045"][columns]
df_hokuyo = df[df['testname'] == 'Hokuyo_4_Hokuyo_8'][columns]

In [ ]:
display_colored(df_bun.sort_values('pcd_err'))

In [ ]:
display_colored(df_hokuyo.sort_values('pcd_err'))

### SHOT

In [ ]:
df = pd.read_csv('data/debug/shot_results_02.csv')
df['pcc'] = df['correct_correspondences'] / df['correspondences']
df_bun = df[df['testname'] == "bun000_bun045"][columns]
df_hokuyo = df[df['testname'] == 'Hokuyo_4_Hokuyo_8'][columns]

In [ ]:
display_colored(df_bun.sort_values('pcd_err'))

In [ ]:
display_colored(df_hokuyo.sort_values('pcd_err'))

In [ ]:
df = pd.read_csv('data/debug/test_results.csv')
df['pcc'] = df['correct_correspondences'] / df['correspondences']
df_chair = df[df['testname'] == 'kig_023_22_chair_kig_022_21_chair'][columns]
display_colored(df_chair)

In [ ]:
def analyze_features(path_src, path_tgt):
    features_src = pd.read_csv(path_src)
    features_tgt = pd.read_csv(path_tgt)
    features_diff = features_src - features_tgt
    f1_diff = features_diff['f1']
    f2_diff = features_diff['f2']
    f3_diff = features_diff['f3']
    f4_diff = features_diff['f4']

    fs1 = f1_diff.values
    f1_std = np.std(fs1)
    print(f'f1: {np.count_nonzero(fs1 > f1_std) / len(fs1)}')

    fs2 = f2_diff.values
    f2_std = np.std(fs2)
    print(f'f2: {np.count_nonzero(fs2 > f2_std) / len(fs2)}')

    fs3 = f3_diff.values
    f3_std = np.std(fs3)
    print(f'f3: {np.count_nonzero(fs3 > f3_std) / len(fs3)}')

    plt.hist(f1_diff, bins='rice')
    plt.show()

    plt.hist(f2_diff, bins='rice')
    plt.show()

    plt.hist(f3_diff, bins='rice')
    plt.show()

In [ ]:
# 5994 - левый угол
# 245 - верх спинки
# 1565 - середина спинки
# 5545 - нижний угол

point_ids = pd.read_csv('data/debug/kig_023_22_chair_kig_022_21_chair_ids_02.csv')
point_ids.set_index('id_src', drop=False, inplace=True)
point_ids_src = point_ids['id_src'].values
point_ids_tgt = point_ids['id_tgt'].values
ids_src = np.array([5994, 245, 1565, 5545])
ids_tgt = np.array([point_ids[point_ids['id_src'] == id_src]['id_tgt'].values[0] for id_src in ids_src])
display(point_ids.loc[ids_src])

In [ ]:
def min_confusion_matrix(hists1, hists2):
    xx, yy = np.meshgrid(np.arange(len(hists1)), np.arange(len(hists2)))
    return np.stack((hists1[xx], hists2[yy]), axis=3).min(axis=3).sum(axis=2)

def l2_confusion_matrix(hists1, hists2):
    xx, yy = np.meshgrid(np.arange(len(hists1)), np.arange(len(hists2)))
    return np.sqrt(((hists1[xx] - hists2[yy]) ** 2).sum(axis=2))
    
def cosine_confusion_matrix(hists1, hists2):
    xx, yy = np.meshgrid(np.arange(len(hists1)), np.arange(len(hists2)))
    return np.sqrt((hists1[xx] * hists2[yy]).sum(axis=2))
    
def show_confusion_matrix(confusion_matrix, labels1=None, labels2=None, vmin=None, vmax=None):
    res = plt.imshow(confusion_matrix, cmap=plt.cm.jet, vmin=vmin, vmax=vmax, interpolation='nearest')
    plt.colorbar(res)
   
    if labels1 is not None: 
        plt.tick_params(labelbottom=False,labeltop=True)
        plt.xticks(range(len(labels1)), labels1, rotation='vertical')
    if labels2 is not None:
        plt.yticks(range(len(labels2)), labels2)
    plt.show()

In [ ]:
def skip_zeros(hist1, hist2):
    mask = ~np.logical_and(hist1 == 0, hist2 == 0)
    return hist1[mask], hist2[mask]

def analyze_descriptors(path_src, path_tgt, ids_src, ids_tgt, descriptor_size, confusion_matrix_fn, vmin=None, vmax=None):
    hists_src = pd.read_csv(path_src, header=None, usecols=np.arange(descriptor_size).tolist())
    hists_tgt = pd.read_csv(path_tgt, header=None, usecols=np.arange(descriptor_size).tolist())
    for id_src, id_tgt in zip(ids_src, ids_tgt):
        show_bar_plots(np.vstack(skip_zeros(hists_src.iloc[id_src].values, hists_tgt.iloc[id_tgt].values)))
   
    show_confusion_matrix(confusion_matrix_fn(hists_src.iloc[ids_src].values, hists_tgt.iloc[ids_tgt].values),
                          ['left upper', 'upper', 'center', 'left bottom'],
                          ['left upper', 'upper', 'center', 'left bottom'],
                          vmin=vmin, vmax=vmax) 
    show_confusion_matrix(confusion_matrix_fn(hists_src.iloc[ids_src].values, hists_src.iloc[ids_src].values),
                          ['left upper', 'upper', 'center', 'left bottom'],
                          ['left upper', 'upper', 'center', 'left bottom'],
                          vmin=vmin, vmax=vmax) 
    show_confusion_matrix(confusion_matrix_fn(hists_tgt.iloc[ids_tgt].values, hists_tgt.iloc[ids_tgt].values),
                          ['left upper', 'upper', 'center', 'left bottom'],
                          ['left upper', 'upper', 'center', 'left bottom'],
                          vmin=vmin, vmax=vmax) 

In [ ]:
analyze_descriptors("data/debug/kig_023_22_chair_kig_022_21_chair_histograms_src_02_shot.csv",
                    "data/debug/kig_023_22_chair_kig_022_21_chair_histograms_tgt_02_shot.csv",
                    ids_src, ids_tgt,
                    descriptor_size=352,
                    confusion_matrix_fn=cosine_confusion_matrix)

In [ ]:
analyze_descriptors("data/debug/kig_023_22_chair_kig_022_21_chair_histograms_src_02.csv",
                    "data/debug/kig_023_22_chair_kig_022_21_chair_histograms_tgt_02.csv",
                    ids_src, ids_tgt,
                    descriptor_size=33,
                    confusion_matrix_fn=min_confusion_matrix)

In [ ]:
df_overlapping  = pd.read_csv('data/kizhi/downsampled/overlapping.csv', index_col='reading')
show_confusion_matrix(df_overlapping.values, df.columns, df.columns, vmin=0.0, vmax=1.0)

In [ ]:
df = pd.read_csv('test_results.csv')
df = df.fillna('')
df['pcc'] = df['correct_correspondences'] / df['correspondences']
df['overlapping'] = [df_overlapping.loc[testname[:testname.rfind('kig') - 1] + '.ply'][testname[testname.rfind('kig'):] + '.ply'] for testname in df['testname']]
df = df[df['overlapping'] > 0.35]
df = df[df['pcd_err'] > 0.1]
df = df[['testname', 'r_err', 't_err', 'pcd_err', 'pcc', 'helpful', 'correct_correspondences', 'correspondences', 'correct_inliers', 'overlapping']].sort_values('pcc')
display_colored(df.sort_values('pcd_err'))